In [1]:
import os

In [4]:
%pwd

'e:\\projects\\Delivery-time-prediction-for-food-devlivery-industry'

In [3]:
os.chdir("../")

In [35]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelRegisterConfig:
  dagshu_tracking_uri : Path
  run_json_info: Path
  repo_owner: str
  repo_name: str

In [36]:
from pathlib import Path

CONFIG_FILE_PATH = Path("E:\projects\Delivery-time-prediction-for-food-devlivery-industry\config\config.yaml")
PARAMS_FILE_PATH = Path("E:\projects\Delivery-time-prediction-for-food-devlivery-industry\params.yaml")
SCHEMA_FILE_PATH = Path("E:\projects\Delivery-time-prediction-for-food-devlivery-industry\schema.yaml")

In [29]:
from Deliveryprediction.constants import *
from Deliveryprediction.utils.common import read_yaml, create_directories

In [37]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_registry_config(self) -> ModelRegisterConfig:
        config = self.config.model_registry    

        #create_directories([config.root_dir])

        model_registry_config = ModelRegisterConfig(
            dagshu_tracking_uri = config.dagshu_tracking_uri,
            run_json_info = config.run_json_info,
            repo_name= config.repo_name,
            repo_owner= config.repo_owner,

        )

        return model_registry_config

In [44]:
import mlflow
import dagshub
import json
from pathlib import Path
from mlflow import MlflowClient
from Deliveryprediction import logger

class MLFlowModelRegistry:
    def __init__(self,logger,config=ModelRegisterConfig):
        self.config = config
        self.repo_owner = config.repo_owner
        self.repo_name = config.repo_name
        self.logger = logger
        self.tracking_uri = config.dagshu_tracking_uri
        self.run_info_path = Path(config.run_json_info)
        self._initialize_dagshub()
        self._set_tracking_uri()
        self.client = MlflowClient()
        

    def _initialize_dagshub(self):
        dagshub.init(repo_owner=self.repo_owner, repo_name=self.repo_name, mlflow=True)

    def _set_tracking_uri(self):
        mlflow.set_tracking_uri(self.tracking_uri)

    def load_model_information(self):
        with open(self.run_info_path) as f:
            return json.load(f)

    def register_and_transition_model(self):
        run_info = self.load_model_information()
        run_id = run_info["run_id"]
        model_name = run_info["model_name"]
        
        model_registry_path = f"runs:/{run_id}/{model_name}"
        model_version = mlflow.register_model(model_uri=model_registry_path, name=model_name)
        
        registered_model_version = model_version.version
        registered_model_name = model_version.name
        
        self.logger.info(f"The latest model version in model registry is {registered_model_version}")
        
        self.client.transition_model_version_stage(
            name=registered_model_name,
            version=registered_model_version,
            stage="Staging"
        )
        
        self.logger.info("Model pushed to Staging stage")


    

In [39]:
config=ConfigurationManager()

[2025-02-19 15:08:46,501: INFO: common: yaml file: E:\projects\Delivery-time-prediction-for-food-devlivery-industry\config\config.yaml loaded successfully]
[2025-02-19 15:08:46,507: INFO: common: yaml file: E:\projects\Delivery-time-prediction-for-food-devlivery-industry\params.yaml loaded successfully]
[2025-02-19 15:08:46,515: INFO: common: yaml file: E:\projects\Delivery-time-prediction-for-food-devlivery-industry\schema.yaml loaded successfully]
[2025-02-19 15:08:46,517: INFO: common: created directory at: artifacts]


In [ ]:
model_registry=config.get_model_registry_config()

In [ ]:
model_register = MLFlowModelRegistry(logger=logger,config=model_registry)

[2025-02-19 15:11:14,876: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as onkar-git

[2025-02-19 15:11:14,887: INFO: helpers: Accessing as onkar-git]
[2025-02-19 15:11:15,790: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/onkar-git/Delivery-time-prediction-for-food-delivery-industry "HTTP/1.1 200 OK"]
[2025-02-19 15:11:16,802: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "onkar-git/Delivery-time-prediction-for-food-delivery-industry"

[2025-02-19 15:11:16,808: INFO: helpers: Initialized MLflow to track repo "onkar-git/Delivery-time-prediction-for-food-delivery-industry"]


Repository onkar-git/Delivery-time-prediction-for-food-delivery-industry initialized!

[2025-02-19 15:11:16,811: INFO: helpers: Repository onkar-git/Delivery-time-prediction-for-food-delivery-industry initialized!]


In [46]:
model_register.register_and_transition_model()

Successfully registered model 'delivery_time_pred_model'.
2025/02/19 15:13:06 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: delivery_time_pred_model, version 1


[2025-02-19 15:13:06,726: INFO: 2349854883: The latest model version in model registry is 1]


Created version '1' of model 'delivery_time_pred_model'.
C:\Users\Admin\AppData\Local\Temp\ipykernel_9400\2349854883.py:44: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  self.client.transition_model_version_stage(


[2025-02-19 15:13:07,057: INFO: 2349854883: Model pushed to Staging stage]
